In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split
path = "/content/drive/MyDrive/TFM/"
path1 = path + "Planets.csv"
path2 = path + "Not_Planets.csv"

In [4]:
#Creación del dataset con los registros de secuencias positivas y secuencias negativas
planets_df = pd.read_csv(path1, names=list(range(4000)))
not_planets_df = pd.read_csv(path2, names=list(range(4000)))
X_data = pd.concat([planets_df, not_planets_df]).to_numpy()
y_data = np.concatenate((np.ones(len(planets_df)), np.zeros(len(not_planets_df))))

#Separación en train y test
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.20, random_state=42)
y_train = np.transpose(np.atleast_2d(y_train))
y_test = np.transpose(np.atleast_2d(y_test))

In [5]:
#Definición del modelo a entrenar
from tensorflow.keras import layers
from tensorflow import keras

model_name = "Conv2D_A"

model = keras.Sequential(
    [
        layers.Input(shape=(4000,1)),
        layers.Reshape((500,8,1)),
        layers.Conv2D(filters=32, kernel_size=(5,2), activation='relu'),
        layers.MaxPooling2D(pool_size=(3,2)),
        layers.Conv2D(filters=32, kernel_size=(5,2), activation='relu'),
        layers.MaxPooling2D(pool_size=(3,2)),
        layers.Dropout(0.2),
        layers.Flatten(),
        layers.Dense(100, activation='sigmoid'),
        layers.Dense(1, activation='sigmoid')
    ],
    name=model_name
)

In [6]:
from datetime import datetime
#Compilación del modelo
model.compile(
    loss=keras.losses.BinaryCrossentropy(),
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    metrics=[
              keras.metrics.BinaryAccuracy(),
              keras.metrics.Recall(),
              keras.metrics.Precision(),
              keras.metrics.AUC(num_thresholds=25)
            ]
    )

#Definición del directorio donde guardar los datos del entrenamiento
logdir = "/content/drive/MyDrive/TFM/Logs/" + model.name + "/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)
model_checkpoint = keras.callbacks.ModelCheckpoint(
    filepath="/content/drive/MyDrive/TFM/Models/" + model_name + "/best_model.keras",
    verbose=1,
    save_best_only=True,
    mode="max",
    monitor="val_binary_accuracy",
)

In [7]:
training_history = model.fit(
    X_train,
    y_train,
    batch_size=64,
    verbose=1, # Suppress chatty output; use Tensorboard instead
    epochs=50,
    validation_data=(X_test, y_test),
    callbacks=[tensorboard_callback, model_checkpoint],
)

Epoch 1/50
520/520 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - auc: 0.6044 - binary_accuracy: 0.5863 - loss: 0.6660 - precision: 0.5612 - recall: 0.7047
Epoch 1: val_binary_accuracy improved from -inf to 0.63617, saving model to /content/drive/MyDrive/TFM/Models/Conv2D_A/best_model.keras
520/520 ━━━━━━━━━━━━━━━━━━━━ 18s 21ms/step - auc: 0.6044 - binary_accuracy: 0.5863 - loss: 0.6660 - precision: 0.5613 - recall: 0.7050 - val_auc: 0.6295 - val_binary_accuracy: 0.6362 - val_loss: 0.6256 - val_precision: 0.5945 - val_recall: 0.9772
Epoch 2/50
517/520 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - auc: 0.6326 - binary_accuracy: 0.6284 - loss: 0.6256 - precision: 0.5877 - recall: 0.9113
Epoch 2: val_binary_accuracy improved from 0.63617 to 0.65880, saving model to /content/drive/MyDrive/TFM/Models/Conv2D_A/best_model.keras
520/520 ━━━━━━━━━━━━━━━━━━━━ 9s 10ms/step - auc: 0.6326 - binary_accuracy: 0.6284 - loss: 0.6255 - precision: 0.5878 - recall: 0.9115 - val_auc: 0.6506 - val_binary_accuracy: 0.6588 - val_l